# Capitulo 2. Proyecto de Machine Learning

👀: verifica que si hayas instalado las librerías que vas a necesitar

En este caso se realizará un modelo para predecir la media de precios en las viviendas de los diferentes municipos de Londres, Reino Unido 🇬🇧.

## <span style="color:green">1. Descargar los datos</span>

Las bases de datos para este proyecto se puede encontrar en este enlace: https://www.kaggle.com/justinas/housing-in-london

También se pueden consultar todas las bases de datos de este curso en GitHub:https://github.com/a2Proyectos/MachineLearning_Data

- housing_in_london_yearly_variables.csv, con los datos que necesitamos para hacer la regresión.
- London_Borough_Excluding_MHW.shp, con los datos que necesitamos para graficar Londres.
- Capitulo_2/housing_in_london_monthly_variables.csv, con los datos de la media salarial

In [ ]:
## Importamos nuestras librerias principales panda, numpy, matplotlib, os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
import kagglehub

# Download latest version
path1 = kagglehub.dataset_download("imanollaconcha/barcelona-fotocasa-housingprices")

print("Path to dataset files:", path1)

In [ ]:
os.listdir(path1)

***

## <span style="color:green">2. Vistazo a la Base de Datos</span>

In [ ]:
import pandas as pd
import os

country_file = os.path.join(path1, 'Barcelona_Fotocasa_HousingPrices.csv')

Barcelona_Housing = pd.read_csv(country_file)

Barcelona_Salary = pd.read_csv("2022_renda_disponible_llars_per_persona.csv")

In [ ]:
Barcelona_Housing

In [ ]:
Barcelona_Salary

In [ ]:
Barcelona_Salary.rename(columns={
    "Any": "Year",
    "Codi_Districte": "District_Code",
    "Nom_Districte": "District_Name",
    "Codi_Barri": "Neighborhood_Code",
    "Nom_Barri": "Neighborhood",
    "Seccio_Censal": "Census_Section",
    "Import_Euros": "Income_Euros"
}, inplace=True)

Barcelona_Housing.rename(columns={
    "price": "Price",
    "rooms": "Rooms",
    "bathroom": "Bathrooms",
    "lift": "Lift",
    "terrace": "Terrace",
    "square_meters": "Square_Meters",
    "real_state": "Property_Type",
    "neighborhood": "Neighborhood",
    "square_meters_price": "Price_per_m2"
}, inplace=True)

In [ ]:
# Limpiar nombres de barrios 
Barcelona_Salary["Neighborhood"] = Barcelona_Salary["Neighborhood"].str.strip().str.lower()
Barcelona_Housing["Neighborhood"] = Barcelona_Housing["Neighborhood"].str.strip().str.lower()

In [ ]:
data = Barcelona_Salary.merge(Barcelona_Housing, on="Neighborhood", how="left")
data

In [ ]:
# Obtener información de los datos.
data.info()

👀:Comienza a ver el tipo de variable, de eso depende el análisis que le demos o si necesitamos cambiarlas a otro tipo de variable

***

In [ ]:
# Una forma muy común para saber que tipo de datos contiene alguna variable de tipo objeto, es contar sus valores, ejemplo:
data["Neighborhood"].value_counts()


In [ ]:
# Una forma muy común para saber que tipo de datos contiene alguna variable de tipo objeto, es contar sus valores, ejemplo:
data["Neighborhood"].value_counts()


In [ ]:
# Una forma muy utilizada para obtener información de nuestros datos numéricos es el método describe(), ejemplo:
pd.options.display.float_format = "{:,.2f}".format
data.describe()

In [ ]:
columns_of_interest = [
    "Neighborhood",
    "Year",
    "Income_Euros",
    "Price",
    "Square_Meters",
    "Price_per_m2",
    "Rooms",
    "Bathrooms",
    "Lift",
    "Terrace",
    "Property_Type"
]

df_filtered = data[columns_of_interest]

# Revisamos las primeras filas
df_filtered.head()

In [ ]:
# Realiza un Histograma para visualizar los datos 
#matplotlib inline para aquellos que estan con jupyter notebook
df_filtered.hist(bins=50, figsize=(15,10))
plt.show()

***

## <span style="color:green">3. Crear un set de entrenamiento y de prueba </span>

Recuerden que nuestro set de prueba lo pondremos de lado por ahora, sin verlo. No sean tramposos ❌

In [ ]:
#Importamos la función para dividir los datos train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
# Seleccionamos por ejemplo, el 30% de los datos para el set de prueba
#Random_state es la semilla que se usa para generar números aleatorios.
set_ent, set_prueba = train_test_split(df_filtered,test_size=0.3,random_state=11)


In [ ]:
#confirmamos la división
print(len(set_ent),len(set_prueba))

Ya tenemos nuestro set de prueba y nuestro set de entrenamiento ✅

***

### <span style="color:blue">3.1 Evitar Sesgo </span>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Crear categorías de renta
df_filtered["income_cat"] = pd.cut(
    df_filtered["Income_Euros"],       # variable continua
    bins=[0, 10000, 20000, 30000, 40000, np.inf],   # 5 niveles
    labels=[1,2,3,4,5]
)

# Histograma de categorías
df_filtered["income_cat"].hist()
plt.title("Distribución de la renta por categoría")
plt.xlabel("Categoría de renta")
plt.ylabel("Cantidad de registros")
plt.show()



In [ ]:
#Verificar que no existen datos en el bin #1
df_filtered["income_cat"].value_counts()

In [ ]:
# Redefinir en 4 categorías, quitando en donde no hay datos (el bin#1)
data = data.dropna(subset=["Income_Euros"])
data = data.reset_index()
data["income_cat"] =pd.cut(data["Income_Euros"],bins=[10000,20000,30000,40000,np.inf],labels=[1,2,3,4])
data["income_cat"].hist()

In [ ]:
#Verifica que si se redefinieron las categorías 
df_filtered["income_cat"].value_counts

In [ ]:
## Dividir datos basándonos en nuestras categorías de salarios
from sklearn.model_selection import StratifiedShuffleSplit


In [ ]:
# Generamos nuestro objeto para que lo divida en 30% y solo haga una división
split = StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=11)

In [ ]:
# Mantener solo categorías con al menos 2 registros
counts = df_filtered["income_cat"].value_counts()
valid_categories = counts[counts >= 2].index
df_filtered = df_filtered[df_filtered["income_cat"].isin(valid_categories)]

In [ ]:
df_filtered = df_filtered.reset_index(drop=True)


In [ ]:
# Creamos nuestras variables basándonos en nuestras categorías
for ent_index, prueba_index in split.split(df_filtered, df_filtered["income_cat"]):#Definir de nuevo, entrenamiento y prueba peroteniedno en cuenta las nuevas etiquetas para eliminar sesgo
    cat_set_ent = df_filtered.loc[ent_index]
    cat_set_prueba = df_filtered.loc[prueba_index]

In [ ]:
# Comprobación. Ya en porcentaje
cat_set_prueba["income_cat"].value_counts()/len(cat_set_prueba)

***

In [ ]:
# Creamos dataframe para trabajar con el set de entrenamiento
# Creamos dataframe para trabajar con el set de entrenamiento
df = cat_set_ent.copy()
df.head()

***

## <span style="color:green">4. Visualizar los Datos Gráficamente </span>


Para esto vamos a necesitar, en conjunto con nuestro dataset LONDON_MAP, una nueva libreria que se llama geopandas, el cual exteiende la libreria pandas, para trabajar con datos geoespaciales, se puede encontrar más información en: https://geopandas.org/getting_started/introduction.html

In [ ]:
#Importar geopandas
import geopandas as gpd

#Leer el mapa
barcelona_map = gpd.read_file("Bc_barris2016_AMB.shp")
barcelona_map

In [ ]:
# Asegúrate de importar pandas, ya que geopandas lo utiliza internamente para concat
import pandas as pd
import geopandas as gpd # Asumo que ya lo tienes importado (gpd.read_file)
import matplotlib.pyplot as plt 
import os
import glob

# Tu código inicial (asumiendo que ya tienes la lista cargada)
# -----------------------------------------------------------------
# Nota: La ruta "\ALÇADES" puede necesitar ser una ruta absoluta o relativa válida
archivos_shp = glob.glob(os.path.join("ALÇADES", '*.shp')) # Eliminando la barra inicial si es relativa

lista_gdf = []

for archivo in archivos_shp:
    print(f"Cargando archivo: {os.path.basename(archivo)}")
    gdf_temp = gpd.read_file(archivo)
    lista_gdf.append(gdf_temp)
# -----------------------------------------------------------------


# PASO CLAVE: Concatenar todos los GeoDataFrames de la lista
# Esto junta todas las filas (entidades) de todos los archivos en un solo objeto.
gdf_barcelona_completo = pd.concat(lista_gdf, ignore_index=True)

print("\n🎉 ¡GeoDataFrames combinados con éxito!")
print(f"Total de entidades (parcelas/subparcelas): {len(gdf_barcelona_completo)}")

In [ ]:
# Configuración básica para el gráfico
fig, ax = plt.subplots(1, 1, figsize=(15, 15))

# Visualizar el GeoDataFrame completo
# `edgecolor='k'` es para ver los límites de cada entidad en color negro.
# `linewidth=0.1` hace que las líneas sean finas.
gdf_barcelona_completo.plot(
    ax=ax, 
    edgecolor='k', 
    linewidth=0.1, 
    color='lightgray' # Color de relleno de las parcelas
)

# Añadir títulos y etiquetas
ax.set_title("Mapa Completo de Barcelona (Combinado de todos los archivos)", fontsize=16)
ax.set_axis_off() # Ocultar ejes de coordenadas para una vista de mapa más limpia

# Mostrar el mapa
plt.show()

***

In [ ]:
# Ajustamos los nombres de las columnas para despúes hacer un merge.
#utilizar lower para cambiar de mayúsculas a minúsculas 



#Seleccionar columnas necesarias


***

In [ ]:
# Seleccionar datos de nuestro set de entrenamiento


# Le asignamos nombre a las columnas del nuevo dataframe y reseteamos el indice


In [ ]:
# Combinar dtaframes


***

In [ ]:
#Gráfica del promedio de los precios en las casas 
#Cuando se grafica en geopandas hay muchos argumentos, no se desesperen si no los recuerdan, es normal. 


💸: Recuerden que los precios son mayores en el centro de la cuidad

In [ ]:
#Graficar ahora el total de las casa vendidas (utilizar el código anterior para no repetir)


**Ejercicio:** Incluir en el análisis los salarios promedio por zona (apoyarse del video 21)

***

## <span style="color:green">5. Medir la Correlación </span>


In [ ]:
# Crear matriz de correlación


# Comparar correlación


In [ ]:
#Importar seaborn 


# Crear vector


# Graficar


In [ ]:
#Importar pandas 


#Para graficar scatter_matrix...


In [ ]:
#Por si quieren ver una gráfica en específico más a detalle


***

## <span style="color:green">6. Combinación de Variables </span>


In [ ]:
#Armar una columna para hacer las combinaciones que necesitamos

# Crear matriz de correlación


***

***

## <span style="color:Blue">7. Transformación de Datos </span>


In [ ]:
# Crear Dataframe de predictores y variable a predecir ✂️

#Nuevo dataframe sin average_price


***

In [ ]:
#Darte una idea de a qué variables le hace faltan datos

#Calcular la suma de todos los vacíos  


In [ ]:
#Quitar el número de crímenes por su alta cantidad de datos vacíos 


In [ ]:
#Tomamos la media 

#Llenamos los valores con la media

#Verifica que no hay datos vacíos


***

Recuerden que volvimos a ejecutar la línea de código de "df = cat_set_ent.drop('average_price', axis=1)" para hacer este ejercicio

In [ ]:
#BIENVENIDOS A SCIKIT <3 Serán unos expertos al final. 
# Rellenar valores con scikit
#1. Recuerden: importar lo que vayas a utiliza, en este caso SimpleImputer


# 2. Crear objeto, en este caso se llamará imputer



# data numérico


#Ejecuta Imputer


#Aplicar transform para rellenar las medianas 


#Regresarlo a dataframe 


***

## <span style="color:Blue">8. Manejo de texto y valores categóricos </span>

In [ ]:
#definir que variable vamos a cambiar a valor numérico

#Convertir variables de texto en numéricas

#verificar que si se haya transformado


#Por si quieres ver como funciona el encoder


In [ ]:
#ONE HOT ENCODER, es importante recordar y entender esta función porque la usaremos en todo el curso
#Convertir variables categóricas en binarias
#Importar OneHotEncoder

#Crear objeto

#Ajustar

#obligar a que nos muestre la matriz 


***

## <span style="color:Blue">9. Escalación de variables </span>


#### <span style="color:Blue">9.1 Normalización </span>

In [ ]:
#Importar MinMaxScaler

#Creamos el objeto 

#Hacer un dataframe con la transformación


#### <span style="color:Blue">9.2 Estandarización </span>

In [ ]:
## Estandarización (Es el más ultizado)
#Importar StandardScaler (presten especial atención a StandardScaler lo estaremos viendo muy seguido) 

#Crea objeto

#Hacer un dataframe con la transformación


***

## <span style="color:Blue">10. Pipeline </span>


In [ ]:
# Crear función de pipeline


In [ ]:
# Column Transformer (nos permite transformar varias columnas al mismo tiempo y luego juntarlas)

#Hacer un datframe que usaremos para la regresión linea

#Visualiza los datos 


***

***

## <span style="color:purple">11. Seleccionar y entrenar modelos </span>

In [ ]:
## Hacer Regresión Lineal (AL FIN)


In [ ]:
#Análisis preambultario de los errores


***

#### <span style="color:purple">11.1 RMSE </span>

In [ ]:
# Calcular el promedio de la suma de los errores al cuadrado RMSE


In [ ]:
#Sacar el promedio 


In [ ]:
#Calcular el porcentaje de acierto


***

#### <span style="color:purple">11.2 Árbol de Decisión </span>

In [ ]:
# importar DecisionTreeRegressor

#crear objeto 

#correrlo


#calcular el error


***

#### <span style="color:purple">11.3 Validación Cruzada </span>

¡Anota esto porque es importante!

In [ ]:
#Importar cross_val_score, creo objeto, corro mi función


In [ ]:
#Calcular el porcentaje de acierto de validación cruzada 


***

#### <span style="color:purple">11.4 Bosque Aleatorio </span>
¡Este también es importante!

In [ ]:
#Importar RandomForestRegressor, creo objeto, corro mi función


#calcular el error


In [ ]:
#Calcular el el promedio de la suma de los errores al cuadrado


In [ ]:
#Calcular el porcentaje de acierto de bosque aleatorio


***

## <span style="color:purple">12. Afinar el modelo </span>

### <span style="color:purple">12.1 Grid Search</span>

In [ ]:
#Importar GridSearchCV


### <span style="color:purple">12.2 set de prueba</span>

In [ ]:
#¡AHORA SI! A utilizar el set de prueba. Es la última parte 

#Define cuál es el modelo final 


#Aquí definimos cuál será nuestra variable a predecir y las predichas 



#Ahora, sí. El pipeline para limpieza de datos 



#Por último, ver el error del modelo 


In [ ]:
#Ver el porcentaje de acierto
